This is an exercise in using convolulation networks with the MNIST dataset.  It will provide some introduction and practice in setting up and assigning parameter values.

The exercise is to run through all the cells - one of the last cells produces images of the filters

Try changing the size of the filters and number of filters in the first convolution layer add statement.  (Look for the 'EXERCISE' comments and the <<<<-----  markers)

Compare how the filters are different when you use 3x3 filters vs something like 16x16

The last cell looks at the convolution layer activation - it is a transformation of the input - check it out and you might see interesting features highlighted


In [ ]:
#  MNIST tutorial (handwritten printed digits recognition tutorial)
#  (more or less from Chollet's book)

# ----------- IMPORT STATEMENTS ---------------
import numpy as np
np.random.seed(1)  # for reproducibility
import matplotlib.pyplot as plt      #These provide matlab type of plotting functions
import matplotlib.image as mpimg

from keras.models import Sequential               #Sequential models are the standard stack of layers models
from keras.layers import Dense, Dropout, Activation, Flatten   #These are core layer specification functions
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D           #These are convolution layer functions
from keras.utils import np_utils                         #Some utilities
from keras import optimizers                             #For training algorithm

from keras import backend as K    #backend is tensorflow

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
#---------------------------------------------
print('import done')

In [ ]:
#--------------- LOAD and PREPARE DATA STATEMENTS ----------------
# Load some numpy arrays that have the MNIST data
#  (these are subsets extracted from the MNIST data set in Keras)
X_train=np.load('X_train5k.npy')
Y_train=np.load('Y_train5k.npy')
X_test =np.load('X_test.npy')
Y_test =np.load('Y_test.npy')

print(X_train.shape)     #review the dimensions Note python3 uses print(X..) python 2 uses print X...



In [ ]:
# --------- Reshape input data ----------------
#  b/c Keras expects N-3D images (ie 4D matrix)
# ----------------------------------------------
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test  = X_test.reshape(X_test.shape[0],   1, 28, 28)

#To confirm, we can print X_train's dimensions again:
print(X_train.shape)

#convert and put into 0-1 range
X_train  = X_train.astype('float32')
X_test   = X_test.astype('float32')
X_train /= 255
X_test  /= 255

# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test  = np_utils.to_categorical(Y_test,  10)

# ------------- End loading and preparing data --------------
print("min Xtrain:"+str(np.amin(X_train)))   #this gets the max value over a flattened numpy array
print("max Xtrain:"+str(np.amax(X_train)))   #this gets the max value over a flattened numpy array
print('prep done')

In [ ]:
# ----------------------------------------------
# --------------Set up Model ---------------------
# ----------------------------------------------
mymodel = Sequential()

numfilters = 32  #<<<< -------- EXERCISE: you can try 8,16,32 for comparison
                 #numbers that divide into 32 are better for GPUs so its often recommended

#     input shape for 1 image, channels refers to color dimension of input image
mymodel.add(Convolution2D(numfilters, (3,3),strides=1,  data_format="channels_first",
                          activation='relu', input_shape=(1,28,28))) 
#                                       /\ 
#                                       /\
#                                       |--------EXERCISE:try filter size to 9,9 16,16 
#                                                      and check accuracy
 
print('modeldef and first conv layer done')


In [ ]:
mymodel.layers[0].output # use this to check sizes of output  

In [ ]:
# ----------------------------------------------
#-----------------Now add more Convolution layers
# ----------------------------------------------
mymodel.add(Convolution2D(numfilters, (3, 3), strides=1, data_format="channels_first", activation='relu'))

mymodel.add(MaxPooling2D(pool_size=(2,2),strides=2,data_format="channels_first")) #get Max over 2D region,and slide
mymodel.add(Dropout(0.25))
 
mymodel.add(Flatten())            #reorganize 2DxFilters output into 1D
print('added more layers')

In [ ]:
mymodel.layers[1].output   #size of output layer for max pooling

In [ ]:
#----------------Now add final classification layers
mymodel.add(Dense(32, activation='relu'))  #enter number of hidden units (no good rule, but start with ~ num of previous output) 


mymodel.add(Dropout(0.25))
mymodel.add(Dense(10, activation='softmax'))
print('assemble model done')

In [ ]:
# --------- Now assemble (ie compile TensorFlow commands) and run -----
mymodel.compile(loss='categorical_crossentropy',
               optimizer='adam',  #NOTE, adam is adative learn.rate, sgd is stoch.grad.descent
                                  #    adam is usually much faster training
               metrics=['accuracy'])
print('compiled')

In [ ]:
mymodel.summary()

In [ ]:
#Now train the model, and use early stopping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

#Stop if there's no improvement after 5 epochs
ES_function    = EarlyStopping(monitor='val_acc', mode='max', patience=5) #or min_delta=1)

#Save model weights at stopping point only
ModCk_function = ModelCheckpoint('es_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#Fit the model, use 20% of data to get validation score
history=mymodel.fit(X_train, Y_train, validation_split=0.20,
          batch_size=32, epochs=50, verbose=1,callbacks=[ES_function,ModCk_function])



In [ ]:
# list all data in history and print out performance
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val_test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val_test'], loc='upper left')
plt.show()

In [ ]:
# ---------------------------------------------------------------------
#--------- Get overall prediction score on training and test sets
# ---------------------------------------------------------------------
from keras.models import load_model
mymodel_best = load_model('es_best_model.h5')
# evaluate the model


trainscore = mymodel_best.evaluate(X_train, Y_train, verbose=1) # get overal score
testscore  = mymodel_best.evaluate(X_test, Y_test, verbose=1) # get overal score
#somepred  = mymodel.predict(X_test,verbose=0)           # use this get prediction values

print('Train Loss: %.3f, Train Acc: %.3f' % (trainscore[0], trainscore[1]))
print('Test Loss: %.3f, Test Acc: %.3f' % (testscore[0], testscore[1]))


In [ ]:
#View the sample of training image ----
img_filename = "Xtrain_num0_cat_5.jpeg" #% scriptDir 
#img_filename = "Xtrain_num1_cat_0.jpeg" #% scriptDir 
im = mpimg.imread(img_filename)
plt.figure()
plt.imshow(im,'gray')
plt.show()
#print('im loaded')

In [ ]:
# ------------ GET WEIGHTS From Convolution Layer and make mosaic image 
Wlist   =mymodel.layers[0].get_weights()      
W3D     =np.squeeze(Wlist[0])
print("W3D shape Wlist[0]:"+str(W3D.shape))
W3Dchan =W3D.swapaxes(1,2).swapaxes(0,1)  #get the channels as 1st dimension;

Wmin       =np.amin(W3Dchan)
Wmax       =np.amax(W3Dchan-Wmin)
Wsc        =np.int_(255*(W3Dchan-Wmin)/Wmax)
ncol =4
nrow =np.ceil(numfilters/ncol)
print(nrow)
plt.figure()
for i in range(Wsc.shape[0]):
   plt.subplot(nrow,ncol,i+1)
   plt.imshow(Wsc[i],'gray')
   plt.axis('off')
#plt.savefig("test.png", bbox_inches='tight')
plt.show()
print('done plotting weights mosaic')

In [ ]:
# ===========================================

In [ ]:
#  ---------------- NOW Visualize the activations for the first training example --------
#   first gather activations from the model layers
# -------------------------------------------------------------------------
import numpy as np
from keras import models   

layer_outputs     = [layer.output for layer in mymodel.layers[:]]
my_model_actvtns  = models.Model(inputs=mymodel.input, outputs=layer_outputs)

x                 = np.expand_dims(X_train[0],0)           #set up a 4D input of 1 image training set 

my_actvtns_output = my_model_actvtns.predict(x)   #for each image get predictions/activatns

print(len(my_actvtns_output))
for act_lay in my_actvtns_output:  #[lay2do]:
        print(act_lay.shape)

In [ ]:
# --- Now output a mosaic of layer 0 output
layeroutput3D      = np.squeeze(my_actvtns_output[0])      
ncol =4
nrow =np.ceil(numfilters/ncol)
plt.figure()
for i in range(layeroutput3D.shape[0]):
   plt.subplot(nrow,ncol,i+1)
   plt.imshow(layeroutput3D[i],'gray')
   plt.axis('off')
#plt.savefig("test.png", bbox_inches='tight')
plt.show()
print('done plotting layer activation output mosaic')
